# 1. DeBug

## 反汇编器`dis`

使用内置的dis模块，查看函数的反汇编代码：

In [2]:
import dis

def add(x, y):
    return x + y + 11*24

dis.dis(add)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 LOAD_CONST               1 (264)
              8 BINARY_ADD
             10 RETURN_VALUE


## 性能测试`timeit`
性能测试，可以直接使用`timeit`模块

In [8]:
import timeit

def foo():
    add(1,2)

# 默认执行100万次
timeit.timeit(setup='from __main__ import foo',stmt = 'foo()', number=1000000)

0.13255862500000148

圈复杂度：代表程序内线性独立路径的数量。值越大表示执行路径越多，逻辑越复杂。

下面的 4.1.1 flake8 就提供了圈复杂度的检测。

也可以通过`pip install radon`按装radon，计算圈复杂度。

# 2. 模块

## 2.1 装饰器

`@wrapt.decorator`函数实现的装饰器，有一个知名的问题。那就是如果装饰类里的方法，因为类方法默认第一个参数是`self`，会出现问题。此时可以使用此第三方库的，解决此问题。

`@functools.wraps(func)`装饰函数，避免元数据覆盖。`functools.update_wrapper(self, function)`类专时期避免元数据覆盖

`@lru_catche(max_size=None)`给函数增加缓存功能：如果某个函数的返回值相对于输入值是固定的，那么可以使用缓存功能，避免重复计算,max_size表示最多保存多少缓存结果。默认为128。


`@property` 将类的方法转化成属性，并可以自定义属性的get set del；

`@abstractmethod` 装饰类方法后，继承此类的类，必须实现被装饰的类方法

`@total_ordering` 装饰一个类，在重载运算符时，只需实现`__eq__`和剩余一个。

`@dataclass`python3.7 后新增内置模块，利用类型注解语法快速定义数据类。

## 2.2 functools

`functools.reduce()` : reduce() 函数的第一个参数是接受两个参数的函数，第二个参数是一个可迭代的对象。假如有个接受两个参数的 $fn$ 函数和一个 lst 列表。调用 reduce(fn, lst) 时，$fn$ 会应用到第一对元素上，即 $fn(lst[0], lst[1])$，生成第一个结果 $r1$。然后，$fn$ 会应用到 $r1$ 和下一个元素 上，即$fn(r1, lst[2])$，生成第二个结果$r2$。接着，调用 $fn(r2, lst[3])$，生成 $r3$……直到最后一个元素，返回最后得到的结果 $rN$。

In [9]:
import functools as func

func.reduce(lambda a, b: a+b, range(1,11))

55

## 2.3 魔术方法

`__init_subclass__` 是类的一个特殊钩子方法，当类在派生出子类时，会触发此方法。

`__getattribute__(self, name)`点属性运算符，调用此方法，如果抛出了Attribute异常，就会去调用`__getattr__`方法


## 2.4 内置函数

`vars(object)`: 可以是模块，类，实例或具有`__dict__`属性的任何对象。返回`__dict__`给定对象的属性。主要是查看实例属性。

`getattr(obj, name [, default])`即obj.name，其实是调用了`__getattribute__(self, name)`魔术方法

# 3. 优秀第三方

## 3.1 AttrDict、addict

Python中如果有嵌套字典的，访问会比较长`dict[attr1][attr2][attr3][attr4]`，使用点号访问能极大缩短访问。

# 4. 代码规范

## 4.1 规范风格统一

### 4.1.1 flake8

> Flake8与运行它的Python版本绑定在一起。如果你希望 Flake8 正确解析Python3.5中的新语言特性，需要将 Flake8 安装在 3.5 版本上。

使用flake8可以轻松检查代码是否遵守了PEP8规范。`E`和`W`开头的错误代码指违反了PEP8规范，`F`开头则来自于代码本身的错误（模块导入没使用，变量定义没使用）。

安装： `python -m pip install flake8`

使用：
```bash
flake8 path/to/code/to/check.py
# or
flake8 path/to/code/

```

### 4.1.2 isort

PEP8规范要求，依赖导入规则分为三组：

1. Python标准库包的导入
2. 第三方包的导入
3. 与当前应用相关的

安装： `pip install isort`

使用： 

要在特定文件上运行：`isort mypythonfile.py mypythonfile2.py` ; 要递归应用：`isort .`

### 4.1.3 black

对Python风格的统一。

### 4.1.4 pre-commit

上述工具安装后，手动执行不论是效率还是可行性都不够。`pre-commit`就是一个钩子功能开发工具。

`pre-commit`文件与项目源码放在一起，都在代码仓储，每个人不需要单独维护，只要安装`pre-commit`工具就行。

使用流程如下：


1. 安装 pre-commit : pip install pre-commit
2. 写.pre-commit-config.yaml配置文件
3. 用pre-commit install安装git hooks到你的.git/目录

配置文件内容如下：

```yaml
fail_fast: true
repos:
- repo:https://github.com/timothycrosley/isort
  rev: 5.7.0
  hooks:
  - id: isort
  	additional_dependencies: [toml]
- repo: https://github.com/psf/black
	rev:20.8b1
	hooks:
	- id: black
		args: [--config=./pyproject.toml]
- repo: https://github.com/pre-commit-hooks
	rev: v2.4.0
	hooks:
	- id: flake8
```

### 4.1.5 mock

unittest.mock 是一个用于测试的Python库。它允许使用模拟对象来替换受测系统的一些部分，并对这些部分如何被使用进行断言判断。

unittest.mock 提供的 Mock 类，能在整个测试套件中模拟大量的方法。创建后，就可以断言调用了哪些方法/属性及其参数。还可以以常规方式指定返回值并设置所需的属性。



模拟方法调用，使用 Mock 的常见场景：

- 模拟函数调用
- 记录在对象上的方法调用

你可能需要替换一个对象上的方法，用于确认此方法被系统中的其他部分调用过，并且调用时使用了正确的参数。

`class Mock(spec=None, side_effect=None, return_value=DEFAULT, wraps=None, name=None, spec_set=None, **kwargs)`Mock类的主要参数有：

- name:命名一个mock对象，只是起到标识作用，可以通过print查看。
- return_value: 定义mock方法的返回值，可以指定一个值（或者对象），当mock对象被调用时，返回return_value指定的值。
- side_effect: 这个参数指向一个可调用对象，接收一个可迭代序列。可以抛出异常或者动态改变值。当传递这个参数的时候return_value 参数就会失效。

下面是用Mock类来模拟一个类实例的某个方法，并使其方法可以返回值。

In [10]:
from unittest.mock import MagicMock


class ProductionClass:
    pass

thing = ProductionClass()
# method绑定到mock上，并且调用会返回9
thing.method = MagicMock(return_value=9)
thing.method(3, 4, 5)

9

In [13]:
# 绑定此方法，调用的时候会抛出异常
thing.err_method = Mock(side_effect=KeyError('foo'))
thing.err_method()

NameError: name 'Mock' is not defined

## 4.2 单元测试

自动化测试分为：UI测试、集成测试、单元测试。

### 4.2.1 unittest

Python中编写单元测试最正统的是使用标准库中的`unittest`模块：
1. 创建继承`unittest.TestCase`的类
2. 编写以`test`开头的测试方法
3. 方法内用`assert`测试断言

`unittest`模块中有不同类型的`assert`供选择使用。


In [7]:
import unittest


def foo_add(x, y):
    return x+y

class TestAdd(unittest.TestCase):
    def test_normal(self):
        self.assertEqual(foo_add(2, 3), 5)
        
if __name__ == '__main__':
    unittest.main()

### 4.2.2 pytest

第三方测试工具`pytest`更加易用。需要安装第三方库：`pip install -U pytest`

1. 创建一个`test_xxx.py`的新文件
2. 包含`test_foo()`的测试函数
3. 直接使用`assert`语句进行判断

In [2]:
# 待测试的函数
def func(x):
    return x + 1

In [ ]:
# 新建一个文件, 导入并书写单元测试
from xxx import func

def test_answer():
    assert func(3) == 5

但是这个测试用例的覆盖率根本不合格，没有考虑其他情况。所以一般的做法是写更多的`test_foo()`函数，pytest为我们提供了更加方便的方式“表驱动测试”

#### 4.2.2.1 parametrize

表驱动测试，是一种简化单元测试代码的技术。它鼓励你用一张包含多分输入参数、期望结果组成的数据表，以此驱动测试执行。

如果要增加测试用例，直接往表里增加一行数据。

利用pytest提供的装饰器`pytest.mark.parametrize`，可以很方便的实现“表驱动测试”：

In [3]:
import pytest

@pytest.mark.parametrize(
    'parament,excepted',
    [
        (5, 6),
        (11, 12),
        (800, 801)
    ]
)
def test_func(parament,excepted):
    assert func(parament) == excepted
    

然后，运行`pytest xxxxx.py`执行上述脚本。其中：

@pytest.mark.parametrize('', [(),(),()])

可以简单分为两个部分，第一部分可以理解为“表驱动测试”表的标头，同时要和装饰器装饰的函数的参数保持一致；

列表嵌套元组即为“表驱动测试”表的行。

#### 4.2.2.2 @pytest.fixture 创建fixture对象

在编写单元测试的时候，我们常会用到一些重复的东西，比如说重复的图片或者重复的对象，这些被称作`fixture`。

下面我们以一个需要重复使用的(伪)token为例：

In [14]:
import pytest

@pytest.fixture
def radom_token() -> str:
    return 'aohdiohh98234h32ih82374yhdwpi'


# 在需要使用的地方，直接写在形参中即可：
def test_foo(radom_token):
    print(radom_token)

之后每次执行`test_foo`，pytest会自己找到形参中的对象。

加入在`@pytest.fixture`装饰的函数中，使用`yield`关键字，就会把它变成一个生成器函数。作用是增加额外的清理逻辑。类似于生命周期管理的形式。

`@pytest.fixture(scope='class')`有五种作用域，可以用 scope 参数来指定其作用域：

- function  ：fixture 默认的作用域，每个测试方法执行一次（如果被调用），默认时可以不用指定；
- class      ：每个测试类执行一次，如果同一个测试类中多次调用该 fixture 也只会执行一次；
- module   ：每个测试模块执行一次，如果同一个测试模块多次调用该 fixture 也只会执行一次；
- package ：（可能淘汰，不建议使用）每个 package 中只执行一次，如果同一个 package 下多次调用，也只会执行一次；
- session   ：每次测试会话中只执行一次，在同一次会话中多次调用，也只会执行一次；

# 写在最后
让我们谨记Python之禅

In [15]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
